# D03: Dataframes - Handling Data

Having learnt how to read data in to Pandas, the next logical step is to learn how to to work with dataframes to convert the dataframe structure and underlying data into something which can be visualised or analysed. This process takes a variety of names as follows:

* Data Cleaning
* Data Cleansing
* Data Wrangling
* Data Munging

These terms all have slightly different meanings but all relate to the same process of 'removing erroneous records and getting data ready for consumption'

We're going to explore this process using pandas now. Firstly we'll need some data...

In [2]:
import pandas as pd

path = "https://vincentarelbundock.github.io/Rdatasets/csv/multgee/arthritis.csv" # Arthritis dataset
df = pd.read_csv(path)
df.head(5)

,Unnamed: 0,id,y,sex,age,trt,baseline,time
0,1,1,4.0,2,54,2,2,1
1,2,1,5.0,2,54,2,2,3
2,3,1,5.0,2,54,2,2,5
3,4,2,4.0,1,41,1,3,1
4,5,2,4.0,1,41,1,3,3


## Basic Data Wrangling

This dataset shows data from a double-blind clinical trial investigating a new treatment for rheumatoid arthritis. However at the moment the column names and data don't make sense to us because it's mostly coded. 

However we can use Pandas to do some basic work on the dataframe to improve this situtation.

In [3]:
new_names = {'Unnamed: 0':'Index',
             'id':'Patient ID',
             'y':'Arthritis Score',
             'sex':'Gender',
             'age':'Age',
             'trt':'Treatment Type',
             'baseline':'Baseline Arthritis Score',
             'time':'Time of Score'}                                   # Dictionary of new column names

order = ['Index','Patient ID','Gender','Age','Treatment Type',
         'Baseline Arthritis Score','Arthritis Score','Time of Score'] # List of the new column order
  
# Refining the structure of the dataset:
    
df = df.rename(columns = new_names)                    # Applying the new names to the dataframe
df = df[order]                                         # Ordering the columns in the dataframe.
df = df.drop(['Index'],axis=1)                         # Dropping a column. The axis=1 is used to specify the vertical (column) axis
df = df.sort_values(by=['Patient ID','Time of Score']) # Sorting the dataframe by 2 x variables
df.head(6)

,Patient ID,Gender,Age,Treatment Type,Baseline Arthritis Score,Arthritis Score,Time of Score
0,1,2,54,2,2,4.0,1
1,1,2,54,2,2,5.0,3
2,1,2,54,2,2,5.0,5
3,2,1,41,1,3,4.0,1
4,2,1,41,1,3,4.0,3
5,2,1,41,1,3,4.0,5


So we've made the dataset make a little more sense through refining the structure, getting rid of unwanted variables and sorting. However some of the data is coded and since we might want to present some of this in tables or charts, it would be good to make it more meaningful to our audience.

## Converting data with functions

One of the ways we can make the data more meaningful is to transform the data to change how it's displayed. We can do this with a function.

In [4]:
# Creating a new column with a function

def gd_change (row):              # Creating the function - the 'row' parameter can be any word. Pandas will automatically assign a value based upon the index
    if row['Gender'] == 1:
        return 'M'
    elif row['Gender'] == 2:
        return 'F'
    else:
        return 'O'
    
df['Gender Desc'] = df.apply(gd_change,axis=1)     # Creating a new column using the apply method to apply the function to the dataframe
df.head(6)

,Patient ID,Gender,Age,Treatment Type,Baseline Arthritis Score,Arthritis Score,Time of Score,Gender Desc
0,1,2,54,2,2,4.0,1,F
1,1,2,54,2,2,5.0,3,F
2,1,2,54,2,2,5.0,5,F
3,2,1,41,1,3,4.0,1,M
4,2,1,41,1,3,4.0,3,M
5,2,1,41,1,3,4.0,5,M


As we can see the lists, dictionaries and functions we learned about in the basics section have come in handy so far!

In this case we've created a 'Gender Desc' variable to describe the gender of the patient. Let's do this for the treatment type too... 

In [5]:
def trt_change (row):              
    if row['Treatment Type'] == 1:
        return 'Treatment'
    elif row['Treatment Type'] == 2:
        return 'Placebo'
    else:
        return 'Other'
    
df['Treatment Desc'] = df.apply(trt_change,axis=1)
df.head(6)

,Patient ID,Gender,Age,Treatment Type,Baseline Arthritis Score,Arthritis Score,Time of Score,Gender Desc,Treatment Desc
0,1,2,54,2,2,4.0,1,F,Placebo
1,1,2,54,2,2,5.0,3,F,Placebo
2,1,2,54,2,2,5.0,5,F,Placebo
3,2,1,41,1,3,4.0,1,M,Treatment
4,2,1,41,1,3,4.0,3,M,Treatment
5,2,1,41,1,3,4.0,5,M,Treatment


## Converting data types

You'll notice that the Baseline Artritis Score and Artritis Score variables are in different formats with one being an integer and one being a float. We might want to change them to match but first we'll need to determine the range of values that exist for both variables. We can do this by using two methods in conjunction with one another:

In [6]:
baseline_values = df['Arthritis Score'].unique()  # Returns the unique values contained in a column
baseline_values

array([  4.,   5.,   3.,   2.,   1.,  nan])

We can see here that there aren't any values to the right of the decimal so we would probably want to convert this variable to an integer. However we can also see a 'Nan' value. Nan stands for 'Not a Number' and is mostly used to denote missing data. If we were to try and convert the entire column to an integer, Pandas would give us an error as it cannot convert Nan values to integers.

This gives us two choices:

1) Write a function using try / except to only convert the floats and handle the exceptions that the Nan values will produce <br/>
2) Convert the Nan values to something that the .astype() method can process. <br/>

In this example we're going to go with option 2 because it's quicker and easier. We can use the fillna() method to determine how pandas will deal with Nan values:


In [7]:
df['Arthritis Score'] = df['Arthritis Score'].fillna(0)     # Replace Nan values with 0's

Before converting the field to an integer with the astype() method:

In [8]:
df['Arthritis Score'] = df['Arthritis Score'].astype('int') # Converts the values in a column to integers
df.head(6)

,Patient ID,Gender,Age,Treatment Type,Baseline Arthritis Score,Arthritis Score,Time of Score,Gender Desc,Treatment Desc
0,1,2,54,2,2,4,1,F,Placebo
1,1,2,54,2,2,5,3,F,Placebo
2,1,2,54,2,2,5,5,F,Placebo
3,2,1,41,1,3,4,1,M,Treatment
4,2,1,41,1,3,4,3,M,Treatment
5,2,1,41,1,3,4,5,M,Treatment


## Method Chaining

In the example above, we converted the df['Arthritis Score'] column from a float to an integer in two steps as follows:

In [ ]:
df['Arthritis Score'] = df['Arthritis Score'].fillna(0)
df['Arthritis Score'] = df['Arthritis Score'].astype('int')

This is inefficient both in terms of time spent typing and also in terms of processing. We can improve both these things with method chaining as follows:

In [9]:
df['Arthritis Score'] = df['Arthritis Score'].fillna(0).astype('int')   # Example of method chaining

Method chaining, according to wikipeida, is:

This allows us to be more efficient with our code and processing which is a good thing. Just remember that you can only use methods that are applicable to that object and as a general rules, Python will execute the methods in the order in which they are chained.

Also note that generally throughout the course we'll not be using much method chaining as it an make things less clear for those who are learning. That said you should always look to chain your methods when writing your own code as it's more efficient in terms of processing and typing.

## Creating new columns

We can also convert data and create some new columns with some simple code we met in the Basics section of the course.

In [10]:
df['Combined Score'] = df['Baseline Arthritis Score'] + df['Arthritis Score']                             # Basic Arithmetic
df['Scores'] = df['Baseline Arthritis Score'].astype('str') + ',' + df['Arthritis Score'].astype('str')   # Using strings
df.head(5)

,Patient ID,Gender,Age,Treatment Type,Baseline Arthritis Score,Arthritis Score,Time of Score,Gender Desc,Treatment Desc,Combined Score,Scores
0,1,2,54,2,2,4,1,F,Placebo,6,"2,4"
1,1,2,54,2,2,5,3,F,Placebo,7,"2,5"
2,1,2,54,2,2,5,5,F,Placebo,7,"2,5"
3,2,1,41,1,3,4,1,M,Treatment,7,"3,4"
4,2,1,41,1,3,4,3,M,Treatment,7,"3,4"


## Further Reading

<a href = "https://en.wikipedia.org/wiki/Method_chaining">Method Chaining</a><br/>